<a href="https://colab.research.google.com/github/peace-and-harmony/image-matting/blob/main/notebooks/modnet_demo_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/Pronti-Beam/demos-experiments/blob/master/notebooks/u_2net_demo_benchmark_edition.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/Pronti-Beam/blob/master/demos-experiments/notebooks/u_2net_demo_benchmark_edition.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td><td>
  <a target="_blank"  href="https://www.linkedin.com/company/pronti-ai/?originalSubdomain=ca">
    <img width=128px src="https://miro.medium.com/max/3402/1*gPSJe7WqcC61cGyB0lxalQ.png" /></a>
</td><td>
  <a target="_blank"  href="http://beamdata.ca/">
    <img width=128px src="http://beamdata.ca/wp-content/uploads/2018/10/beamdata_web_logo.png" /></a>
</td></table>


# MODNet benchmark demo

In this demo, the differece between Pytorch and ONNX runtime is compared.

**Note** Use Runtime: CPU as base of the benchmard

  - pytorch inference:
    - inference cost for batch_size 1: 478.1471ms
  - ONNX Runtime Inference:
    - inference cost for batch_size 1: 359.8167 ms

**Note** Use Runtime: GPU as TensorRT optimize

  - modnet-cpu-simplified.onnx inference:
    - Latency: min = 7.54675 ms, max = 8.21542 ms, mean = 7.71543 ms, median = 7.70303 ms, percentile(99%) = 7.98169 ms
    - GPU Compute Time: min = 7.14722 ms, max = 7.77158 ms, mean = 7.26397 ms, median = 7.25128 ms, percentile(99%) = 7.53656 ms
  - modnet-gpu-simplified.onnx inference:
    - Latency: min = 7.58453 ms, max = 8.2428 ms, mean = 7.76476 ms, median = 7.75586 ms, percentile(99%) = 8.04858 ms
    - GPU Compute Time: min = 7.19623 ms, max = 7.84821 ms, mean = 7.32274 ms, median = 7.30762 ms, percentile(99%) = 7.57153 ms

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


<p align="justify">Clone the repository, and download the pre-trained model:</p>

### Pronti MODNet demo

In [ ]:
%cd /content
!ls /content/drive/MyDrive/Cropper
!cp /content/drive/MyDrive/Cropper/cropper_validation.zip /content
!unzip cropper_validation.zip

In [18]:
%cd /content/valid_validation
!mkdir test_run
!cp -r image test_run

/content/valid_validation


In [2]:
%cd /content
!git clone https://github.com/ZHKKKe/MODNet.git

/content
Cloning into 'MODNet'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 249 (delta 21), reused 24 (delta 9), pack-reused 206
Receiving objects: 100% (249/249), 60.76 MiB | 29.50 MiB/s, done.
Resolving deltas: 100% (82/82), done.


#### Download the pre-trained MODNet model

In [3]:
import os
%cd /content/MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt

/content/MODNet
Downloading...
From: https://drive.google.com/uc?id=1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz
To: /content/MODNet/pretrained/modnet_photographic_portrait_matting.ckpt
26.3MB [00:00, 56.6MB/s]


In [19]:
import os
import glob
!mkdir -p /content/MODNet/test_data/test_human_images
base = '/content/MODNet/test_data/test_human_images'
img_path = '/content/valid_validation/test_run/image'
 
files = glob.glob(f'{base}/*')
 
for f in files:
    os.remove(f)
 
!cp {img_path}/* {base}

In [20]:
!mkdir -p /content/MODNet/test_data/test_human_images_results/
%ls /content/MODNet/test_data/

test_human_images/  test_human_images_results/


#### alpha matte predition

MODNet **inference** using pre-trained .ckpt

In [ ]:
import os
import sys
import argparse
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from src.models.modnet import MODNet

# check input arguments
input_path = '/content/MODNet/test_data/test_human_images'
output_path = '/content/MODNet/test_data/test_human_images_results'
ckpt_path = '/content/MODNet/pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(input_path):
  print('Cannot find input path: {0}'.format(input_path))
  exit()
if not os.path.exists(output_path):
  print('Cannot find output path: {0}'.format(output_path))
  exit()
if not os.path.exists(ckpt_path):
  print('Cannot find ckpt path: {0}'.format(ckpt_path))
  exit()

# define hyper-parameters
ref_size = 512

# define image to tensor transform
im_transform = transforms.Compose(
  [
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ]
)

# create MODNet and load the pre-trained ckpt
modnet = MODNet(backbone_pretrained=False)
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('using GPU!')
  modnet = nn.DataParallel(modnet).cuda()
else:
  device = torch.device('cpu')
  modnet = nn.DataParallel(modnet)
  print('using CPU!')

modnet.load_state_dict(torch.load(ckpt_path, map_location=device))
modnet.eval() # turn the model to inference model

# inference images
im_names = os.listdir(input_path)
for im_name in im_names:
  print('Process image: {0}'.format(im_name))

  # read image
  im = Image.open(os.path.join(input_path, im_name))

  # unify image channels to 3
  im = np.asarray(im)
  if len(im.shape) == 2:
      im = im[:, :, None]
  if im.shape[2] == 1:
      im = np.repeat(im, 3, axis=2)
  elif im.shape[2] == 4:
      im = im[:, :, 0:3]

  # convert image to PyTorch tensor
  im = Image.fromarray(im)
  im = im_transform(im)

  # add mini-batch dim
  im = im[None, :, :, :]

  # resize image for input
  im_b, im_c, im_h, im_w = im.shape
  if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
      if im_w >= im_h:
          im_rh = ref_size
          im_rw = int(im_w / im_h * ref_size)
      elif im_w < im_h:
          im_rw = ref_size
          im_rh = int(im_h / im_w * ref_size)
  else:
      im_rh = im_h
      im_rw = im_w
  
  im_rw = im_rw - im_rw % 32
  im_rh = im_rh - im_rh % 32
  im = F.interpolate(im, size=(im_rh, im_rw), mode='area')

  # inference
  if torch.cuda.is_available():
    _, _, matte = modnet(im.cuda(), True)
  else:
    _, _, matte = modnet(im, True)

  # resize and save matte
  matte = F.interpolate(matte, size=(im_h, im_w), mode='area')
  matte = matte[0][0].data.cpu().numpy()
  matte_name = im_name.split('.')[0] + '.png'
  Image.fromarray(((matte * 255).astype('uint8')), mode='L').save(os.path.join(output_path, matte_name))


Now simply go ahead and choose a resultant predicton from the dropdown below, the observation widget will recognise a change and toggle a concetanated ground-truth/predicted matte visualization.

In [ ]:
from IPython.display import display,  clear_output
from ipywidgets import Dropdown, Output
 
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
 
images = glob.glob('test_data/test_human_images_results/*')
out = Output()
 
def viz(img, save_path=None, resize=0.5):
  imgo = glob.glob(f'{img_path}/{os.path.basename(os.path.splitext(img)[0])}*')
  img = cv2.imread(img)
  org_img = cv2.imread(imgo[0])

  vis_concat = np.concatenate((img, org_img), axis=1)
  result = cv2.resize(vis_concat, (0,0), fx=resize, fy=resize) 
  
  if save_path is not None:
    cv2.imwrite(f'{save_path}/{os.path.basename(imgo[0])}', result)
  else:
    cv2_imshow(result)
 
 
def dropdown_eventhandler(change):
    with out:
        clear_output()
        print('loading...')

        viz(change.new, resize=0.75)
 
dropdown = Dropdown(description="Choose one:", options=images)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)
display(out)

Dropdown(description='Choose one:', options=('test_data/test_human_images_results/004537.png', 'test_data/test…

Output()

### Concatenate result and save to Mydrive

In [ ]:
from tqdm.notebook import tqdm
!mkdir -p test_data/test_human_images_results_concat
save_path = 'test_data/test_human_images_results_concat'

for i in tqdm(images):
  viz(i, save_path, resize=1.0)

!cp -r test_data/test_human_images_results_concat /content/drive/MyDrive/Beamdata-Pronti/test_human_images_results_concat-modnet

# Training

While it's not immediately clear that fine-tuning MODNet is a possibility without overfitting it's perfectly possible that combining any labelled images of clothing available with either the `DUTS-TR` geheral object saliency dataset of the `supervisely human dataset` would yield a model that generalises to both domains given the results above 


In [ ]:
!mkdir -p train_data/DUTS/DUTS-TR
%cd train_data/DUTS/DUTS-TR
!wget http://saliencydetection.net/duts/download/DUTS-TR.zip
!unzip DUTS-TR.zip && rm DUTS-TR.zip

# Generate .onnx
- Optimisation

ValueError: Unsupported ONNX opset version: 13

  - opset_version=12



The next step should we decide to continue and pursuse this avenue would be to investigate whether or need this particular graph is rejected by TesnorRT's onnnx parser, or alternatively coreml etc should you want to deploy directly on the edge.

#### install the requirements for converting MODNet to .onnx

In [4]:
%cd /content/MODNet
!pip install -r onnx/requirements.txt

/content/MODNet
     |████████████████████████████████| 14.5 MB 6.9 MB/s 
     |████████████████████████████████| 4.1 MB 51.3 MB/s 
     |████████████████████████████████| 50.4 MB 27 kB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0

In [8]:
%ls /content/drive/MyDrive/Beamdata-Pronti/modelsv2/

_bcev2_itr_101297_tar_29.000000.pth  _bcev2_itr_41916_tar_12.000000.pth
_bcev2_itr_104790_tar_30.000000.pth  _bcev2_itr_45409_tar_13.000000.pth
_bcev2_itr_10479_tar_3.000000.pth    _bcev2_itr_48902_tar_14.000000.pth
_bcev2_itr_108283_tar_31.000000.pth  _bcev2_itr_52395_tar_15.000000.pth
_bcev2_itr_111776_tar_32.000000.pth  _bcev2_itr_55888_tar_16.000000.pth
_bcev2_itr_115269_tar_33.000000.pth  _bcev2_itr_59381_tar_17.000000.pth
_bcev2_itr_118762_tar_34.000000.pth  _bcev2_itr_62874_tar_18.000000.pth
_bcev2_itr_122255_tar_35.000000.pth  _bcev2_itr_66367_tar_19.000000.pth
_bcev2_itr_125748_tar_36.000000.pth  _bcev2_itr_69860_tar_20.000000.pth
_bcev2_itr_129241_tar_37.000000.pth  _bcev2_itr_6986_tar_2.000000.pth
_bcev2_itr_132734_tar_38.000000.pth  _bcev2_itr_73353_tar_21.000000.pth
_bcev2_itr_136472_tar_39.000000.pth  _bcev2_itr_76846_tar_22.000000.pth
_bcev2_itr_13972_tar_4.000000.pth    _bcev2_itr_80339_tar_23.000000.pth
_bcev2_itr_140210_tar_40.000000.pth  _bcev2_itr_83832_tar_24.00000

In [9]:
!cp /content/drive/MyDrive/Beamdata-Pronti/modelsv2/_bcev2_itr_158900_tar_45.000000.pth /content/MODNet/pretrained/

### Export to cpu-based .onnx

In [10]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet_photographic_portrait_matting.ckpt
modnet-gpu.onnx                      README.md


In [11]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable

from onnx import modnet_onnx

# define model & load checkpoint
modnet = modnet_onnx.MODNet(backbone_pretrained=False)

# prepare dummy_input
batch_size = 1
height = 512
width = 512

# dummy_input: input tensor x. The values in this can be random as long as it is the right type and size
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('using gpu!')
  dummy_input = Variable(torch.randn(batch_size, 3, height, width)).cuda()
  modnet = nn.DataParallel(modnet).cuda()

else:
  device = torch.device('cpu')
  print('using cpu')
  dummy_input = Variable(torch.randn(batch_size, 3, height, width))
  modnet = nn.DataParallel(modnet)

state_dict = torch.load(ckpt_path, map_location=device)
modnet.load_state_dict(state_dict)
modnet.eval() # set the model to inference mode

# general input
input_name = '/content/MODNet/pretrained/_bcev2_itr_158900_tar_45.000000.pth'

# check input arguments
if not os.path.exists(input_name):
    print('Cannot find checkpoint path: {0}'.format(input_name))
    exit()

if torch.cuda.is_available():
  output_name = '/content/MODNet/pretrained/modnet-45-gpu.onnx'
else:
  output_name = '/content/MODNet/pretrained/modnet-cpu.onnx'


# export to onnx model
torch.onnx.export(
    modnet.module, dummy_input, output_name, export_params = True, 
    input_names = ['input'], output_names = ['output'], 
    dynamic_axes = {'input': {0:'batch_size', 2:'height', 3:'width'}, 'output': {0: 'batch_size', 2: 'height', 3: 'width'}}, opset_version=12)

%ls /content/MODNet/pretrained/

using gpu!


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


_bcev2_itr_158900_tar_45.000000.pth  modnet_photographic_portrait_matting.ckpt
modnet-45-gpu.onnx                   README.md
modnet-gpu.onnx


.onnx is a binary protobuf file which contains both the network structure and parameters of the model

Save modnet-cpu.onnx to drive for further onnx simplifier later

In [70]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet-gpu-simplified.onnx
modnet-45-gpu.onnx                   modnet_photographic_portrait_matting.ckpt
modnet-45-gpu-simplified.onnx        README.md
modnet-gpu.onnx


In [72]:
!cp /content/MODNet/pretrained/modnet-45-gpu-simplified.onnx /content/drive/MyDrive/Beamdata-Pronti/models/

In [73]:
%ls /content/drive/MyDrive/Beamdata-Pronti/models/

_bce_itr_101297_tar_29.000000.pth  _bce_itr_69860_tar_20.000000.pth
_bce_itr_104790_tar_30.000000.pth  _bce_itr_6986_tar_2.000000.pth
_bce_itr_10479_tar_3.000000.pth    _bce_itr_73353_tar_21.000000.pth
_bce_itr_108283_tar_31.000000.pth  _bce_itr_76846_tar_22.000000.pth
_bce_itr_111776_tar_32.000000.pth  _bce_itr_80339_tar_23.000000.pth
_bce_itr_115269_tar_33.000000.pth  _bce_itr_83832_tar_24.000000.pth
_bce_itr_13972_tar_4.000000.pth    _bce_itr_87325_tar_25.000000.pth
_bce_itr_17465_tar_5.000000.pth    _bce_itr_90818_tar_26.000000.pth
_bce_itr_20958_tar_6.000000.pth    _bce_itr_94311_tar_27.000000.pth
_bce_itr_24451_tar_7.000000.pth    _bce_itr_97804_tar_28.000000.pth
_bce_itr_27944_tar_8.000000.pth    logs/
_bce_itr_31437_tar_9.000000.pth    mobilenetv2_human_seg.ckpt
_bce_itr_34930_tar_10.000000.pth   modnet-45-gpu.onnx
_bce_itr_3493_tar_1.000000.pth     modnet-45-gpu-simplified.onnx
_bce_itr_38423_tar_11.000000.pth   modnet-cpu.onnx
_bce_itr_41916_tar_12.000000.pth   modnet-cpu-onn

### Output of PyTorch and ONNX match test
  - trace-based means that it operates by executing your model once
  - the output of PyTorch and ONNX Runtime runs match numerically with the given precision (rtol=1e-03 and atol=1e-05)
  - onnx exporter is correct

In [15]:
import onnxruntime
import numpy as np


# Input to the model
x = torch.randn(1, 3, 512, 512, requires_grad=True)
torch_out = modnet(x)

ort_session = onnxruntime.InferenceSession("/content/MODNet/pretrained/modnet-45-gpu.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


Exported model has been tested with ONNXRuntime, and the result looks good!


#ONNX Runtime

Just as we've seen with ESNet I'm providing data with a dataloader except here we're using the custom data loader provided, along transforms shown in the U-2-Net repo including rescaling to 320x320

In [16]:
!pip install onnxruntime
!pip install onnx

In [21]:
import glob
import os

%cd /content/MODNet/
img_name_list = glob.glob('/content/valid_validation/test_run/image/' + os.sep + '*')

/content/MODNet


In [22]:
import torchvision
from torchvision import transforms

val_data = "/content/valid_validation/test_run"

TRANSFORM_IMG = transforms.Compose([
     transforms.Resize((512, 512)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

val = torchvision.datasets.ImageFolder(val_data, transform=TRANSFORM_IMG)

In [23]:
# ONNX Runtime Inference

from torch.utils.data import Dataset, DataLoader

import onnxruntime as rt  
import time
from tqdm.notebook import tqdm

n_runs = 200

sess_options = rt.SessionOptions()

sess_options.intra_op_num_threads = 4
sess_options.execution_mode = rt.ExecutionMode.ORT_SEQUENTIAL
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

# To enable model serialization and store the optimized graph to desired location.
#sess_options.optimized_model_filepath = '/content/esnet/optimized/esnet.onnx'
session = rt.InferenceSession('/content/MODNet/pretrained/modnet-gpu.onnx', sess_options=sess_options)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dynamic = False
if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=4)
        batch = next(iter(dataloader))
        batch = tuple(t.to('cpu') for t in batch)

        start = time.time()
        pred = session.run([output_name], {input_name: batch[0].numpy()})[0]
        end = time.time()
        runtimes.append((end-start)*1000)

        #print(pred.shapredictions, bitmask = torch.max(pred_torch, 1)pe)
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)} ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

  0%|          | 0/200 [00:00<?, ?it/s]

inference cost for batch_size 1: 510.3938 ms
overall inference execution cost: 184.8089 seconds


### dynamic = True, 
  - batch_size test

In [ ]:
# ONNX Runtime Inference

from torch.utils.data import Dataset, DataLoader

import onnxruntime as rt  
import time
from tqdm.notebook import tqdm

n_runs = 200

sess_options = rt.SessionOptions()

sess_options.intra_op_num_threads = 4
sess_options.execution_mode = rt.ExecutionMode.ORT_SEQUENTIAL
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

# To enable model serialization and store the optimized graph to desired location.
#sess_options.optimized_model_filepath = '/content/esnet/optimized/esnet.onnx'
session = rt.InferenceSession('/content/MODNet/pretrained/modnet-cpu.onnx', sess_options=sess_options)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dynamic = True
if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=4)
        batch = next(iter(dataloader))
        batch = tuple(t.to('cpu') for t in batch)

        start = time.time()
        pred = session.run([output_name], {input_name: batch[0].numpy()})[0]
        end = time.time()
        runtimes.append((end-start)*1000)

        #print(pred.shapredictions, bitmask = torch.max(pred_torch, 1)pe)
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)} ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")


inference cost for batch_size 1: 539.3214 ms



inference cost for batch_size 2: 1119.0548 ms



inference cost for batch_size 4: 1967.1371 ms



inference cost for batch_size 8: 3915.729 ms


KeyboardInterrupt: ignored

#### modnet-cpu.onnx
  - inference cost for batch_size 1: 539.3214 ms
  - inference cost for batch_size 2: 1119.0548 ms
  - inference cost for batch_size 4: 1967.1371 ms
  - inference cost for batch_size 8: 3915.729 ms


#Pytorch comparison




In [24]:
import time
from tqdm.notebook import tqdm

dynamic = False


if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

n_runs = 200

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=4)
        batch = next(iter(dataloader))
        # batch = batch['image'].type(torch.FloatTensor)
        batch = tuple(t.to('cpu') for t in batch)
        # input = Variable(batch)

        #print(batch)
        start = time.time()
        # d1,d2,d3,d4,d5,d6,d7= net(input)
        d1 = modnet(batch[0])
        end = time.time()
        runtimes.append((end-start)*1000)

        #print(pred.shape)
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)}ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

  0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


inference cost for batch_size 1: 21.385ms
overall inference execution cost: 82.4358 seconds


### dynamic = True, 
  - batch_size test

In [ ]:
import time
from tqdm.notebook import tqdm

dynamic = True


if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

n_runs = 200

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        dataloader = DataLoader(dataset=val, batch_size=batch_size, shuffle=True, num_workers=4)
        batch = next(iter(dataloader))
        # batch = batch['image'].type(torch.FloatTensor)
        batch = tuple(t.to('cpu') for t in batch)
        # input = Variable(batch)

        #print(batch)
        start = time.time()
        # d1,d2,d3,d4,d5,d6,d7= net(input)
        d1 = modnet(batch[0])
        end = time.time()
        runtimes.append((end-start)*1000)

        #print(pred.shape)
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)}ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "



inference cost for batch_size 1: 655.4115ms



inference cost for batch_size 2: 1326.4062ms



inference cost for batch_size 4: 2622.8371ms



inference cost for batch_size 8: 5196.7031ms


KeyboardInterrupt: ignored

#### modnet-cpu.onnx
  - inference cost for batch_size 1: 655.4115ms
  - inference cost for batch_size 2: 1326.4062ms
  - inference cost for batch_size 4: 2622.8371ms
  - inference cost for batch_size 8: 5196.7031ms


### Using of data_loader.py

In [28]:
!cp /content/drive/MyDrive/script/data_loader.py .

In [29]:
import torchvision
from torchvision import transforms
from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset
from tqdm.notebook import tqdm
 
 
test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                        lbl_name_list = [],
                                        transform=transforms.Compose([RescaleT(512),
                                                                      ToTensorLab(flag=0)]))

#### Pytorch inference

In [ ]:
import time
from tqdm.notebook import tqdm


if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)

n_runs = 100
dynamic = False

test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                          batch_size=1,
                                          shuffle=True,
                                        num_workers=1)

start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        batch = next(iter(test_salobj_dataloader))
        batch = batch['image'].type(torch.FloatTensor)
        input = Variable(batch)

        #print(batch)
        start = time.time()
        d1 = modnet(input)
        end = time.time()
        runtimes.append((end-start)*1000)

        #print(pred.shape)
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)}ms")

end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3658: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  



inference cost for batch_size 1: 478.1471ms
overall inference execution cost: 81.467 seconds


#### ONNX Runtime Inference


In [30]:
 from torch.utils.data import Dataset, DataLoader
 
import onnxruntime as rt  
import time
import numpy as np
from tqdm.notebook import tqdm
 
n_runs = 100
 
sess_options = rt.SessionOptions()
 
sess_options.intra_op_num_threads = 4
sess_options.execution_mode = rt.ExecutionMode.ORT_SEQUENTIAL
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL
 
# To enable model serialization and store the optimized graph to desired location.
#sess_options.optimized_model_filepath = '/content/esnet/optimized/esnet.onnx'
session = rt.InferenceSession('/content/MODNet/pretrained/modnet-gpu.onnx', sess_options=sess_options)
 
input_name = session.get_inputs()[0].name
print(input_name) 
 
if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)
 
test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                          batch_size=1,
                                          shuffle=True,
                                        num_workers=1)
start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        batch = next(iter(test_salobj_dataloader))
        start = time.time()
        # d1,d2,d3,d4,d5,d6,d7 = session.run(None, {input_name: batch['image'].cpu().numpy().astype(np.float32)})
        d1 = session.run(None, {input_name: batch['image'].cpu().numpy().astype(np.float32)})
        end = time.time()
        runtimes.append((end-start)*1000)
 
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)} ms")
 
end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

input


  0%|          | 0/100 [00:00<?, ?it/s]

inference cost for batch_size 1: 508.65 ms
overall inference execution cost: 82.5082 seconds


In [39]:
from PIL import Image
from skimage import io, transform
from google.colab.patches import cv2_imshow
import cv2

np.seterr(divide='ignore', invalid='ignore')

def normPRED(d):    
  ma = np.argmax(d, 0)
  mi = np.argmin(d, 0)
  dn = (d-mi)/(ma-mi)
  return dn

pred = d1[0][:,0,:,:]
predict = normPRED(pred)
predict = pred.squeeze()

im = Image.fromarray(predict*255).convert('RGB')
idx = batch['imidx'].numpy()[0][0]
image_name = img_name_list[idx]

 
img_name = image_name.split(os.sep)[-1]
image = io.imread(image_name)
imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)
imo.save('test.png')


IMG = cv2.imread('test.png')
 
vis_concat = np.concatenate((IMG, image), axis=1)
result = cv2.resize(vis_concat, (0,0), fx=0.75, fy=0.75)
 
cv2_imshow(result)

IndexError: ignored

### onnxsim modnet-cpu.onnx

In [67]:
%cd /content/
#Otherwise we get a random pad node error
!pip3 install -U pip && pip3 install onnx-simplifier

/content


In [69]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet-gpu-simplified.onnx
modnet-45-gpu.onnx                   modnet_photographic_portrait_matting.ckpt
modnet-45-gpu-simplified.onnx        README.md
modnet-gpu.onnx


In [68]:
!python3 -m onnxsim /content/MODNet/pretrained/modnet-45-gpu.onnx /content/MODNet/pretrained/modnet-45-gpu-simplified.onnx --input-shape 1,3,512,512

Simplifying...
Note: The input shape of the simplified model will be overwritten by the value of '--input-shape' argument. Pass '--dynamic-input-shape' if it is not what you want. Run 'python3 -m onnxsim -h' for details.
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!


In [63]:
!du /content/MODNet/pretrained/modnet-gpu.onnx

25284	/content/MODNet/pretrained/modnet-gpu.onnx


In [64]:
!du /content/MODNet/pretrained/modnet-gpu-simplified.onnx

25276	/content/MODNet/pretrained/modnet-gpu-simplified.onnx


In [52]:
 from torch.utils.data import Dataset, DataLoader
 
import onnxruntime as rt  
import time
import numpy as np
from tqdm.notebook import tqdm
 
n_runs = 100
 
sess_options = rt.SessionOptions()
 
sess_options.intra_op_num_threads = 4
sess_options.execution_mode = rt.ExecutionMode.ORT_SEQUENTIAL
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL
 
# To enable model serialization and store the optimized graph to desired location.
#sess_options.optimized_model_filepath = '/content/esnet/optimized/esnet.onnx'
session = rt.InferenceSession('/content/MODNet/pretrained/modnet-gpu-simplified.onnx', sess_options=sess_options)
 
input_name = session.get_inputs()[0].name
print(input_name) 
 
if dynamic:
  bsize = (1,2,4,8,16,32,64)
else:
  bsize = (1,)
 
test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                          batch_size=1,
                                          shuffle=True,
                                        num_workers=1)
start_full = time.time()
for batch_size in bsize:
    runtimes = []
    for _ in tqdm(range(n_runs)):
        batch = next(iter(test_salobj_dataloader))
        start = time.time()
        # d1,d2,d3,d4,d5,d6,d7 = session.run(None, {input_name: batch['image'].cpu().numpy().astype(np.float32)})
        d1 = session.run(None, {input_name: batch['image'].cpu().numpy().astype(np.float32)})
        end = time.time()
        runtimes.append((end-start)*1000)
 
    print(f"inference cost for batch_size {batch_size}: {round(sum(runtimes)/len(runtimes), 4)} ms")
 
end_full = time.time()
overall_cost = (end_full - start_full)
print(f"overall inference execution cost: {round(overall_cost, 4)} seconds")

input


  0%|          | 0/100 [00:00<?, ?it/s]

inference cost for batch_size 1: 513.5926 ms
overall inference execution cost: 82.9328 seconds


### onnxsim modnet-gpu.onnx

In [ ]:
!cp /content/drive/MyDrive/Beamdata-Pronti/models/modnet-gpu.onnx /content

In [ ]:
!python3 -m onnxsim /content/modnet-gpu.onnx /content/modnet-gpu-simplified.onnx --input-shape 1,3,512,512

Simplifying...
Note: The input shape of the simplified model will be overwritten by the value of '--input-shape' argument. Pass '--dynamic-input-shape' if it is not what you want. Run 'python3 -m onnxsim -h' for details.
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!


# Model Visualization

In [59]:
!pip install -q netron

     |████████████████████████████████| 1.3 MB 8.3 MB/s 


## Model Visualization via Netron

In [57]:
%ls /content/MODNet/pretrained/

_bcev2_itr_158900_tar_45.000000.pth  modnet-gpu-simplified.onnx
modnet-45-gpu.onnx                   modnet_photographic_portrait_matting.ckpt
modnet-gpu.onnx                      README.md


In [61]:
import netron
import portpicker
from google.colab import output

port = portpicker.pick_unused_port()

# Read the model file and start the netron browser.
with output.temporary():
  netron.start('/content/MODNet/pretrained/modnet-gpu.onnx', address=port, browse=True)

output.serve_kernel_port_as_iframe(port, height='800')

<IPython.core.display.Javascript object>

In [62]:
import netron
import portpicker
from google.colab import output

port = portpicker.pick_unused_port()

# Read the model file and start the netron browser.
with output.temporary():
  netron.start('/content/MODNet/pretrained/modnet-gpu-simplified.onnx', address=port, browse=True)

output.serve_kernel_port_as_iframe(port, height='800')

<IPython.core.display.Javascript object>